In [2]:
import requests
import os
import pandas as pd
import time

# Call API function

In [3]:
def get_json_data(Season):
    MeasureType = 'Base'
    SeasonTypes = ['Pre%20Season','PlayIn','Playoffs','Regular%20Season']
    headers= {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36',
              'Referer': 'https://www.nba.com/'}
    res_list = []
    for SeasonType in SeasonTypes:
        print(Season,SeasonType,MeasureType)
        url = f'https://stats.nba.com/stats/teamgamelogs?DateFrom=&DateTo=&GameSegment=&ISTRound=&LastNGames=0&LeagueID=00&Location=&MeasureType={MeasureType}&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=Totals&Period=0&PlusMinus=N&Rank=N&Season={Season}&SeasonSegment=&SeasonType={SeasonType}&ShotClockRange=&VsConference=&VsDivision='

        reps = requests.get(url, headers=headers)
        if reps.status_code == 200:
            jsonData = reps.json()

            rows = jsonData['resultSets'][0]['rowSet']
            columns = jsonData['resultSets'][0]['headers']

            res = pd.DataFrame(rows, columns=columns)
            res = res[[x for x in res.columns if 'RANK' not in x]]
            res.drop(columns=['AVAILABLE_FLAG'],inplace=True)
            res['season_type'] = SeasonType.replace("%20"," ")
            res_list.append(res)
        else:
            print('error')
            reps.raise_for_status()
        time.sleep(1)

    return pd.concat(res_list)

# Get List of Seasons

In [4]:
def format_season(x):
    second = str(int(str(x)[-2:]) + 1)[-2:]
    if len(second) <2:
        second = '0'+second
    return str(x) + '-' + second

seasons = [format_season(x) for x in range(1996,2024)]

# Get the Data

In [5]:
# check folder for seasons already present
available_seasons = [x for x in os.listdir('data') if x.endswith('.csv')]
available_seasons = [x.replace('full_season_','').replace('.csv','') for x in available_seasons]
seasons = [x for x in seasons if x not in available_seasons]
seasons

[]

In [6]:
for season in seasons:
    res = get_json_data(season)
    res.to_csv(f'data/full_season_{season}.csv',index=False)

In [7]:
available_seasons

['1996-97',
 '1997-98',
 '1998-99',
 '1999-00',
 '2000-01',
 '2001-02',
 '2002-03',
 '2003-04',
 '2004-05',
 '2005-06',
 '2006-07',
 '2007-08',
 '2008-09',
 '2009-10',
 '2010-11',
 '2011-12',
 '2012-13',
 '2013-14',
 '2014-15',
 '2015-16',
 '2016-17',
 '2017-18',
 '2018-19',
 '2019-20',
 '2020-21',
 '2021-22',
 '2022-23',
 '2023-24']

In [8]:
available_seasons = [x for x in os.listdir('data') if x.endswith('.csv')]
all_data = []

for season in available_seasons:
    temp = pd.read_csv('data/' + season)
    all_data.append(temp)
all_data = pd.concat(all_data).drop_duplicates().sort_values(['SEASON_YEAR','GAME_DATE']).to_csv('data/all_box_scores.csv')

In [9]:
all_data